In [1]:
import importlib

import model
import pan_loader
import base_config
import loss_functions as L

import utils

import torch
import torch.nn as nn
import torch.optim as optim

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

# train_img_dir = val_img_dir 
# train_seg_dir = val_seg_dir 
# train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
config = base_config.Config()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [7]:
net = model.hgmodel()
# model_dir="models/"
# model_name="first_0.pt"
# pretrained_dict = torch.load(model_dir+model_name)
# net_dict = net.state_dict()

# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
# net_dict.update(pretrained_dict) 
# net.load_state_dict(net_dict)


In [8]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)

set_trainable(net.mb0,True)
set_trainable(net.mb1,True)
set_trainable(net.cb,True)

set_trainable(net.iresnet0, False)
set_trainable(net.iresnet1, False)

for name,module in net.iresnet0.named_modules():
    if 'copy' in name:
        set_trainable(module, True)
    if isinstance(module,nn.BatchNorm2d):
        set_trainable(module, False)
        module.eval()
        
param_lr = []
param_lr.append({'params': net.mb0.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.mb1.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.cb.parameters(),'lr':1e-3,'momentum':0.9})

for name,module in net.iresnet0.named_modules():
    if 'copy' in name:
        param_lr.append({'params':module.parameters(),'lr':1e-3,'momentum':0.9})

param_lr = [{'params':net.parameters(),'lr':1e-3, 'momentum':0.9}]
net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-2, eps=1e-08)

111115750 59594934
[{'params': <generator object Module.parameters at 0x7f5dccb8fd58>, 'lr': 0.001, 'momentum': 0.9}]


In [9]:
# net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()

In [10]:
ckpt = utils.Checkpoint(iters_per_epoch=60, model_dir="./models/", model_name="first")

for i, data in enumerate(train_loader,0):
#     print("batch %d:"%i)
    optimizer.zero_grad()
    
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    del(data)
    outs = net([images,impulses])
    del(images, impulses)
    loss = L.loss_criterion1(outs, [instance_masks, cat_ids])
    del(instance_masks, cat_ids, outs)
    ckpt.update(loss.data, net)
    loss.backward()
    del(loss)
    optimizer.step()

torch.Size([3, 1, 448, 448]) torch.Size([3, 1, 448, 448]) torch.Size([3]) torch.Size([3, 134])
tensor(0.9017, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.9058, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9839, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.8962, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8993, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.8676, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9559, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.9290, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([3, 1, 448, 448]) torch.Size([3, 1, 448, 448]) torch.Size([3]) torch.Size([3, 134])
tensor(0.9429, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.639

torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9017, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.3438, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([2, 1, 448, 448]) torch.Size([2, 1, 448, 448]) torch.Size([2]) torch.Size([2, 134])
tensor(0.8475, device='cuda:0', grad_fn=<MeanBackward1>) tensor(5.3110, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8666, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.7820, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9163, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.5748, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9673, device='cuda:0', grad_fn=<MeanBackward1>) tensor(5.908

torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9286, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.1231, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9121, device='cuda:0', grad_fn=<MeanBackward1>) tensor(5.0938, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9489, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.6925, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8682, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.6252, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9165, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.451

torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8985, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.9202, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9191, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.8911, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9072, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.7845, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8869, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.8893, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9115, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.041

tensor(0.9284, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.6460, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9093, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.9284, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9209, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.6707, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8997, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.9727, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9364, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.1010, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448

...

tensor(0.9210, device='cuda:0', grad_fn=<MeanBackward1>) tensor(1.9646, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9299, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.2326, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9453, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.9389, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9058, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.3335, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(2.9273, device='cuda:0')
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8999, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.3469, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 

tensor(0.8477, device='cuda:0', grad_fn=<MeanBackward1>) tensor(0.9904, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9215, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.5825, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9214, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.2519, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9213, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.7663, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9036, device='cuda:0', grad_fn=<MeanBackward1>) tensor(1.9150, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448

torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8830, device='cuda:0', grad_fn=<MeanBackward1>) tensor(1.8147, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9862, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.1389, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9217, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.8838, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.8658, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.9012, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9305, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.257

tensor(0.8211, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.1900, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9192, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.5549, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9159, device='cuda:0', grad_fn=<MeanBackward1>) tensor(3.8464, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9391, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.8891, device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4, 1, 448, 448]) torch.Size([4, 1, 448, 448]) torch.Size([4]) torch.Size([4, 134])
tensor(0.9204, device='cuda:0', grad_fn=<MeanBackward1>) tensor(2.0594, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(2.8650, device='cuda:0')
torch.Size([4, 1, 

RuntimeError: expected a non-empty list of Tensors